## 1. Download a local copy of an opensourced LLM

In [1]:
import os
import torch
from huggingface_hub import hf_hub_download

<font size="2">Prerequisites: create access token on huggingFace via profile settings and set this token in your system user variables </font>

In [2]:
HUGGING_FACE_API_KEY = os.environ.get("HUGGING_FACE_API_KEY") 

The opensource model [**BERT**](https://huggingface.co/google-bert/bert-base-uncased?text=This+review+expresses+a+%5BMASK%5D+sentiment+about+the+movie.+The+review%3A+This+movie+left+me+very+neutral) has a transformer architecture, meaning it uses self-attention mechanisms to understand the relationships between words in a sentence, regardless of their distance. It is a smaller and faster version of the well-known BERT (Bidirectional Encoder Representations from Transformers) model. It was created by Google Ai
* BERT is pretrained on a large corpus of English data in a self-supervised fashion. This means it was pretrained on the raw texts only, with no humans labeling them in any way

In [17]:
model_id = "google-bert/bert-base-uncased"
filenames = ["config.json","flax_model.msgpack","model.onnx","model.safetensors","pytorch_model.bin","rust_model.ot","tf_model.h5","tokenizer.json",
             "tokenizer_config.json","vocab.txt"]


<font size="2"> Donwload the model so it runs locally </font>

In [18]:
for filename in filenames: 
    downloaded_model_path = hf_hub_download(
            repo_id = model_id,
            filename = filename,
            token = HUGGING_FACE_API_KEY
    )
    print (downloaded_model_path) 

C:\Users\Gordts-De Laender\.cache\huggingface\hub\models--google-bert--bert-base-uncased\snapshots\86b5e0934494bd15c9632b12f734a8a67f723594\config.json
C:\Users\Gordts-De Laender\.cache\huggingface\hub\models--google-bert--bert-base-uncased\snapshots\86b5e0934494bd15c9632b12f734a8a67f723594\flax_model.msgpack
C:\Users\Gordts-De Laender\.cache\huggingface\hub\models--google-bert--bert-base-uncased\snapshots\86b5e0934494bd15c9632b12f734a8a67f723594\model.onnx
C:\Users\Gordts-De Laender\.cache\huggingface\hub\models--google-bert--bert-base-uncased\snapshots\86b5e0934494bd15c9632b12f734a8a67f723594\model.safetensors
C:\Users\Gordts-De Laender\.cache\huggingface\hub\models--google-bert--bert-base-uncased\snapshots\86b5e0934494bd15c9632b12f734a8a67f723594\pytorch_model.bin
C:\Users\Gordts-De Laender\.cache\huggingface\hub\models--google-bert--bert-base-uncased\snapshots\86b5e0934494bd15c9632b12f734a8a67f723594\rust_model.ot
C:\Users\Gordts-De Laender\.cache\huggingface\hub\models--google-ber

[DistilBertForSequenceClassification](https://huggingface.co/docs/transformers/main/en/model_doc/distilbert#transformers.DistilBertForSequenceClassification) is built upon the DistilBERT model and goes beyond the core DistilBERT model by adding a classification head on top of the encoder's output. This classification head takes the encoded representation of the sequence and transforms it into a probability distribution over different predefined classes. See [config.json](https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english/blob/main/config.json) for the labels.

In [46]:
from transformers import BertTokenizer, BertForMaskedLM, pipeline
tokenizer = BertTokenizer.from_pretrained(model_id)
model = BertForMaskedLM.from_pretrained(model_id)

unmasker = pipeline('fill-mask', model=model, tokenizer=tokenizer)

Some weights of the model checkpoint at google-bert/bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


The following will print out the label and the corresponding confidence score (using the softmax function). The confidence score can be interpreted as a measure of how certain the model is about its prediction. For more info see [TextClassificationPipeline](https://huggingface.co/docs/transformers/main_classes/pipelines)

## 2. Prompt engineering strategies

In [47]:
unmasker("This review expresses a [MASK] sentiment about the movie. The review: This movie left me very neutral")

[{'score': 0.563468337059021,
  'token': 3893,
  'token_str': 'positive',
  'sequence': 'this review expresses a positive sentiment about the movie. the review : this movie left me very neutral'},
 {'score': 0.07307648658752441,
  'token': 2844,
  'token_str': 'strong',
  'sequence': 'this review expresses a strong sentiment about the movie. the review : this movie left me very neutral'},
 {'score': 0.05969291180372238,
  'token': 2204,
  'token_str': 'good',
  'sequence': 'this review expresses a good sentiment about the movie. the review : this movie left me very neutral'},
 {'score': 0.0392783023416996,
  'token': 3056,
  'token_str': 'certain',
  'sequence': 'this review expresses a certain sentiment about the movie. the review : this movie left me very neutral'},
 {'score': 0.03506943956017494,
  'token': 4997,
  'token_str': 'negative',
  'sequence': 'this review expresses a negative sentiment about the movie. the review : this movie left me very neutral'}]

In [27]:
classifier("Walking out of a disappointing movie, you might mutter: 'The movie was a total letdown. The acting was terrible, and the plot was predictable.' In the end, I felt...")

[{'label': 'NEGATIVE', 'score': 0.9998030066490173}]

In [32]:
classifier("The book was well-written and thought-provoking, but the ending left me wanting more. (Neutral)" )

[{'label': 'NEGATIVE', 'score': 0.969277560710907}]

In [31]:
classifier("After a wonderful dining experience, a customer would say: "This restaurant was fantastic! The food was incredible, 
and the service was impeccable. Overall, the experience was...")

SyntaxError: unterminated string literal (detected at line 2) (3468017004.py, line 2)

In [8]:
classifier("After a wonderful dining experience, a customer would say: "This restaurant was fantastic! The food was incredible, 
and the service was impeccable. Overall, the experience was...")

[{'label': 'NEGATIVE', 'score': 0.9920676946640015}]

In [9]:
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
with torch.no_grad(): #Disabling gradient calculation improves efficiency for inference tasks.
    logits = model(**inputs).logits #The model processes the tokenized input and outputs a tensor named logits.logits represent the unnormalized scores (before applying softmax) for each potential sentiment class.

predicted_class_id = logits.argmax().item() #This corresponds to the class with the highest predicted probability.
model.config.id2label[predicted_class_id] #refers to the configuration of the pre-trained model (see config.json)

'POSITIVE'

##### 2.1 adding instruction

In [15]:
sentiment_prompt = "Identify a list of emotions that the writer of the following review is expressing:"

In [16]:
classifier(sentiment_prompt + "I love this product! It’s so amazing and convenient.")

[{'label': 'POSITIVE', 'score': 0.9998377561569214}]

##### 2.2 zero-shot 

##### 2.3 few-shot 

## 3. Stress-testing the model

##### 3.1 Inputs that should work well

##### 3.2 Failure-case inputs

##### 3.3 Test for consistency (looping???)

##### 3.4 Test for reliability (score, testing on the applied function to the model outputs in order to retrieve the scores,... )